Following this tutorial https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/


In [2]:
# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Claire\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [4]:
# spacy for lemmatization
import spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','http', 'rt', 'easi', 'co', 'opensource','open','source','likalo','llc','sourc'])

In [7]:
df = pd.read_csv(r'C:\Users\Claire\hackstir\data_opensource.csv')

In [8]:
df.head()

,Unnamed: 0,index,id,coordinates,created_at,user,timestamp_ms,text,retweet_count,retweeted_status,reply_count,entities,open source,#opensource
0,10,10,1.000030e+18,NaN,2018-05-25 15:04:04.000,"{'id': 734197130, 'id_str': '734197130', 'name...",2018-05-25 15:04:04.847,@AaronHawryluk @elonmusk You didn't even let h...,0.0,NaN,0.0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",1,0
1,10003,3,1.000224e+18,NaN,2018-05-26 03:55:12.000,"{'id': 147808626, 'id_str': '147808626', 'name...",2018-05-26 03:55:12.639,RT @ClouDatAI: Adams39Antonio : Turn-key Wire!...,0.0,{'created_at': 'Sat May 26 03:42:07 +0000 2018...,0.0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",1,0
2,10007,7,1.000224e+18,NaN,2018-05-26 03:55:54.000,"{'id': 61550705, 'id_str': '61550705', 'name':...",2018-05-26 03:55:54.921,RT @lagergren: So Oracle executed another deat...,0.0,{'created_at': 'Fri May 25 07:12:19 +0000 2018...,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",1,0
3,10010,10,1.000224e+18,NaN,2018-05-26 03:57:03.000,"{'id': 2707237027, 'id_str': '2707237027', 'na...",2018-05-26 03:57:03.510,RT @rikkiends: 15 books for kids who (you want...,0.0,{'created_at': 'Fri May 25 10:39:17 +0000 2018...,0.0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",1,0
4,10012,12,1.000224e+18,NaN,2018-05-26 03:57:13.000,"{'id': 4461171220, 'id_str': '4461171220', 'na...",2018-05-26 03:57:13.284,Networks like Gnutella and Tor seem to be open...,0.0,NaN,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",1,0


In [9]:
data = df.text.values.tolist()
data = [re.sub(r"@\S+|https?://\S+", '', tweet) for tweet in data]
pprint(data[:1])

["  You didn't even let him respond...lol.\r\n"
 '\r\n'
 "Making it open source wouldn't really mean shit, s… "]


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['you', 'didn', 'even', 'let', 'him', 'respond', 'lol', 'making', 'it', 'open', 'source', 'wouldn', 'really', 'mean', 'shit']]


In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=150)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[5]]])

['rt', 'going', 'to', 'tuck', 'this', 'manafort_filing_right', 'here', 'open', 'source', 'link', 'to', 'page_document', 'in', 'the', 'tweet_below']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [14]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [15]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['even', 'let', 'respond', 'lol', 'make', 'really', 'mean', 'shit']]


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


A slight digression to look at what we've got ... 

In [17]:
id2word[0]

'even'

In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('even', 1),
  ('let', 1),
  ('lol', 1),
  ('make', 1),
  ('mean', 1),
  ('really', 1),
  ('respond', 1),
  ('shit', 1)]]

In [19]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('even', 1),
  ('let', 1),
  ('lol', 1),
  ('make', 1),
  ('mean', 1),
  ('really', 1),
  ('respond', 1),
  ('shit', 1)]]

Ok, let's get back to the topic modelling. 
We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.056*"today" + 0.047*"software" + 0.043*"beautiful_map" + 0.022*"free" + '
  '0.020*"framework" + 0.018*"geospatial" + 0.018*"officially_launche" + '
  '0.015*"decentralize" + 0.015*"first" + 0.014*"introduce"'),
 (1,
  '0.072*"tool" + 0.029*"http" + 0.028*"amazing" + 0.026*"work" + '
  '0.020*"develop" + 0.018*"buy" + 0.015*"safe" + 0.015*"use" + '
  '0.015*"hardware_ledger" + 0.014*"amazing_customer"'),
 (2,
  '0.067*"help" + 0.065*"make" + 0.059*"developer" + 0.058*"management" + '
  '0.049*"pentester" + 0.041*"perform_scan" + 0.034*"good" + 0.020*"come" + '
  '0.020*"blockchain" + 0.014*"best"'),
 (3,
  '0.052*"start" + 0.042*"look" + 0.024*"get" + 0.023*"security" + '
  '0.022*"base" + 0.021*"ai" + 0.021*"go" + 0.021*"code" + 0.018*"build" + '
  '0.016*"system"'),
 (4,
  '0.169*"project" + 0.077*"find" + 0.075*"easy" + 0.074*"technology" + '
  '0.073*"well" + 0.073*"improve" + 0.072*"hire" + 0.072*"company" + '
  '0.071*"sponsor" + 0.071*"marketing"')]


In [39]:
# print(doc_lda[1])
test = lda_model[corpus]
test = [x for x in test]

KeyboardInterrupt: 

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -3.8202821721297635

Coherence Score:  0.4365245104932173


#Visualising the topics
Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

In [31]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.440337 -0.027451       1        1  79.480316
1      0.131249 -0.211055       2        1   6.195778
0      0.116761  0.003810       3        1   5.108829
3      0.110862 -0.134404       4        1   4.710472
2      0.081465  0.369100       5        1   4.504610, topic_info=     Category          Freq              Term         Total  loglift  logprob
30    Default  38233.000000           project  38233.000000  30.0000  30.0000
87    Default  17529.000000              find  17529.000000  29.0000  29.0000
86    Default  17074.000000              easy  17074.000000  28.0000  28.0000
83    Default  16734.000000        technology  16734.000000  27.0000  27.0000
93    Default  16630.000000              well  16630.000000  26.0000  26.0000
...       ...           ...               ...           ...      ...      ...
238    Topic5     68.176567             small     69.377197   3.0826  -5.2395
609    Topic5     63.232880          database     64.369659   3.0823  -5.3148
246    Topic5    158.135101               see    172.001831   3.0160  -4.3982
4894   Topic5     97.654305  patent_landscape    105.796700   3.0200  -4.8802
4895   Topic5     97.654305         safeguard    105.796700   3.0200  -4.8802

[203 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
4886      1  0.005275  actionable_insight
4886      2  0.052749  actionable_insight
4886      3  0.938934  actionable_insight
116       1  0.003486                  ai
116       4  0.996908                  ai
...     ...       ...                 ...
167       1  0.010676               write
167       4  0.992897               write
1230      1  0.978005           xpsviewer
15        1  0.013942                year
15        3  0.989912                year

[337 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 4, 3])

In [53]:
import os
from gensim.models.wrappers import LdaMallet
os.environ['MALLET_HOME'] = 'C:/Users/Claire/hackstir/mallet-2.0.8/mallet-2.0.8'


done that


So far you have seen Gensim’s inbuilt version of the LDA algorithm. Mallet’s version, however, often gives a better quality of topics.

Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself.

In [54]:
# C:\Users\Claire\hackstir
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'C:/Users/Claire/hackstir/mallet-2.0.8/mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [55]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(7,
  [('project', 0.1338710991607594),
   ('improve', 0.070887313615253),
   ('tool', 0.06860588283223336),
   ('win', 0.06697628941579076),
   ('easy', 0.06461337896194899),
   ('company', 0.06388006192454983),
   ('hire', 0.05996903772508759),
   ('seo', 0.04725820907683533),
   ('sponsor', 0.04131019310681985),
   ('technology', 0.03968059969037725)]),
 (4,
  [('sponsor', 0.10742909987669544),
   ('win', 0.09918310727496918),
   ('improve', 0.09394266337854501),
   ('seo', 0.093788532675709),
   ('company', 0.09170776818742293),
   ('find', 0.08901048088779284),
   ('technology', 0.08546547472256473),
   ('easy', 0.08238286066584463),
   ('project', 0.07945437731196055),
   ('hire', 0.07359741060419235)]),
 (17,
  [('improve', 0.10914228977316123),
   ('sponsor', 0.10173375085923776),
   ('company', 0.09745665622851905),
   ('marketing', 0.09295043152829756),
   ('hire', 0.0885969602077446),
   ('seo', 0.08210494157183228),
   ('technology', 0.07973726418697014),
   ('find', 0.077

May we should see that the optimal number of ould see that the optimal number of topics is ... 

In [56]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [57]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [1]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'plt' is not defined

Looks like maybe 30 topics would be better?

In [59]:
ldamallet30 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=30, id2word=id2word)
# Show Topics
pprint(ldamallet30.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet30 = CoherenceModel(model=ldamallet30, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet30 = coherence_model_ldamallet30.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet30)

[(12,
  [('project', 0.16588401033106362),
   ('win', 0.09098379901385302),
   ('technology', 0.08288330594036158),
   ('seo', 0.07536980511857244),
   ('find', 0.0736088283634656),
   ('hire', 0.07337403146278469),
   ('improve', 0.07102606245597558),
   ('sponsor', 0.065977929091336),
   ('marketing', 0.0630429678328246),
   ('easy', 0.060929795726696406)]),
 (26,
  [('seo', 0.10508664058133035),
   ('win', 0.1034097261039687),
   ('improve', 0.1009502515371716),
   ('hire', 0.09726103968697597),
   ('technology', 0.0946897708216881),
   ('sponsor', 0.09044158747903856),
   ('easy', 0.08865287870318614),
   ('marketing', 0.0771380659586361),
   ('company', 0.07613191727221912),
   ('project', 0.06730016769144774)]),
 (21,
  [('win', 0.11303370786516855),
   ('seo', 0.11235955056179775),
   ('sponsor', 0.10404494382022472),
   ('marketing', 0.09741573033707865),
   ('technology', 0.08853932584269662),
   ('project', 0.08573033707865169),
   ('find', 0.08303370786516855),
   ('improve'

Let's visualise this topic model with 30 topics ... 

In [65]:
# Visualize the topics
#pyLDAvis.enable_notebook()
#vis2 = pyLDAvis.gensim.prepare(ldamallet30, corpus, id2word)
#vis2

# above doesn't work, seems the mallet model has to be converted to lda first
mallet2lda_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet30)
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim.prepare(mallet2lda_model, corpus, id2word)
vis2

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7     -0.075787 -0.004937       1        1  3.402231
20    -0.072103  0.002569       2        1  3.398894
5     -0.072592 -0.000069       3        1  3.398651
16    -0.072499  0.001685       4        1  3.398647
6     -0.073696 -0.002370       5        1  3.398638
22    -0.073596 -0.000447       6        1  3.398101
15    -0.072231  0.001100       7        1  3.397083
25    -0.073948 -0.002712       8        1  3.392916
26    -0.074365 -0.004035       9        1  3.391033
13    -0.072326  0.000676      10        1  3.389578
14    -0.073991 -0.003213      11        1  3.385347
3     -0.071880  0.002346      12        1  3.385298
18    -0.073590 -0.002659      13        1  3.385211
21    -0.072310 -0.000705      14        1  3.382614
29    -0.073387 -0.001983      15        1  3.372712
4     -0.069758  0.001438      16        1  3.372253
1     -0.071000  0.001580      17        1  3.370949
23    -0.067277  0.004062      18        1  3.365545
28    -0.071541 -0.003880      19        1  3.354632
12    -0.064273  0.009922      20        1  3.347775
19    -0.068719 -0.004365      21        1  3.346850
10    -0.066688  0.002567      22        1  3.338998
2     -0.066826  0.002148      23        1  3.336916
17    -0.066122  0.002976      24        1  3.315168
11    -0.060367 -0.000920      25        1  3.312097
8     -0.032484 -0.042513      26        1  3.231396
0      0.493888  0.256270      27        1  3.139638
9      0.484039 -0.033942      28        1  3.127777
27     0.347107  0.192439      29        1  3.090602
24     0.478322 -0.373029      30        1  3.072453, topic_info=     Category          Freq      Term         Total  loglift  logprob
79    Default   1174.000000  software   1174.000000  30.0000  30.0000
88    Default  20650.000000   improve  20650.000000  29.0000  29.0000
90    Default  20557.000000       seo  20557.000000  28.0000  28.0000
86    Default  20679.000000      find  20679.000000  27.0000  27.0000
91    Default  20632.000000   sponsor  20632.000000  26.0000  26.0000
...       ...           ...       ...           ...      ...      ...
556   Topic30     91.716216       top     92.825014   3.4707  -4.5527
1121  Topic30     79.705759   publish     80.843943   3.4685  -4.6931
343   Topic30     51.317407    manage     52.328839   3.4632  -5.1334
10    Topic30    331.925355      tool   1087.703973   2.2958  -3.2665
160   Topic30     87.348778      read    146.534303   2.9653  -4.6015

[2142 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1533      7  0.929461     ableist
4557     14  0.928845  absolutely
3200     16  0.918718         abt
3223     18  0.947159    abundant
3223     24  0.045103    abundant
...     ...       ...         ...
4474     10  0.915896         xml
15       29  1.000246        year
99        8  0.978234   years_ago
2697     22  0.904608        zend
3568      6  0.915054       zodat

[2348 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 21, 6, 17, 7, 23, 16, 26, 27, 14, 15, 4, 19, 22, 30, 5, 2, 24, 29, 13, 20, 11, 3, 18, 12, 9, 1, 10, 28, 25])